<a href="https://colab.research.google.com/github/mulargui/Kaggle-Walmart-fcst-tensorflow/blob/master/src/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

print(pd.__version__)

0.22.0


In [2]:
#load data
dftrain=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/train.csv')
dftest=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/test.csv')

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
421570
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
115064


In [3]:
#Add Seasonality Column based on Date
dftrain['Seasonality'] = pd.to_datetime(dftrain['Date']).apply(lambda x: x.weekofyear)
dftest['Seasonality'] = pd.to_datetime(dftest['Date']).apply(lambda x: x.weekofyear)

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday  Seasonality
0      1     1  2010-02-05      24924.50      False            5
1      1     1  2010-02-12      46039.49       True            6
2      1     1  2010-02-19      41595.55      False            7
3      1     1  2010-02-26      19403.54      False            8
4      1     1  2010-03-05      21827.90      False            9
421570
   Store  Dept        Date  IsHoliday  Seasonality
0      1     1  2012-11-02      False           44
1      1     1  2012-11-09      False           45
2      1     1  2012-11-16      False           46
3      1     1  2012-11-23       True           47
4      1     1  2012-11-30      False           48
115064


In [4]:
#Add Year column
dftrain['Year'] = pd.to_datetime(dftrain['Date']).apply(lambda x: x.year)
dftest['Year'] = pd.to_datetime(dftest['Date']).apply(lambda x: x.year)

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday  Seasonality  Year
0      1     1  2010-02-05      24924.50      False            5  2010
1      1     1  2010-02-12      46039.49       True            6  2010
2      1     1  2010-02-19      41595.55      False            7  2010
3      1     1  2010-02-26      19403.54      False            8  2010
4      1     1  2010-03-05      21827.90      False            9  2010
421570
   Store  Dept        Date  IsHoliday  Seasonality  Year
0      1     1  2012-11-02      False           44  2012
1      1     1  2012-11-09      False           45  2012
2      1     1  2012-11-16      False           46  2012
3      1     1  2012-11-23       True           47  2012
4      1     1  2012-11-30      False           48  2012
115064


In [5]:
#adjust sales in weeks 47, 48 and 49 in 2011 to have same number of sale days in week 49 than in 2012
#filter rows affected
dfchristmas = dftrain[(dftrain.Year == 2011) & \
  ((dftrain.Seasonality == 47) | (dftrain.Seasonality == 48) | (dftrain.Seasonality == 49))]

print(dfchristmas.head())
print(dfchristmas.shape[0])

     Store  Dept        Date  Weekly_Sales  IsHoliday  Seasonality  Year
94       1     1  2011-11-25      20911.25       True           47  2011
95       1     1  2011-12-02      25293.49      False           48  2011
96       1     1  2011-12-09      33305.92      False           49  2011
237      1     2  2011-11-25      44259.59       True           47  2011
238      1     2  2011-12-02      45928.14      False           48  2011
9025


In [6]:
#adjust only those with all 3 weeks of sales
dfchristmas = dfchristmas.groupby(['Store', 'Dept']).filter(lambda x: len(x) == 3 )

print(dfchristmas.head())
print(dfchristmas.shape[0])

     Store  Dept        Date  Weekly_Sales  IsHoliday  Seasonality  Year
94       1     1  2011-11-25      20911.25       True           47  2011
95       1     1  2011-12-02      25293.49      False           48  2011
96       1     1  2011-12-09      33305.92      False           49  2011
237      1     2  2011-11-25      44259.59       True           47  2011
238      1     2  2011-12-02      45928.14      False           48  2011
8793


In [7]:
#adjust sales in weeks 47, 48 and 49 in 2011
dfgrouped = dfchristmas.groupby(['Store', 'Dept'])
for name, group in dfgrouped:
  inc_sales = group['Weekly_Sales'].sum() / 21
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2011) & (dftrain.Seasonality == 47),'Weekly_Sales'] -= inc_sales
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2011) & (dftrain.Seasonality == 48),'Weekly_Sales'] -= inc_sales
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2011) & (dftrain.Seasonality == 49),'Weekly_Sales'] += (2*inc_sales)

print(dftrain.head())
print(dftrain.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday  Seasonality  Year
0      1     1  2010-02-05      24924.50      False            5  2010
1      1     1  2010-02-12      46039.49       True            6  2010
2      1     1  2010-02-19      41595.55      False            7  2010
3      1     1  2010-02-26      19403.54      False            8  2010
4      1     1  2010-03-05      21827.90      False            9  2010
421570


In [8]:
#load features
dfstores=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/stores.csv')
dffeatures=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/features.csv')

print(dfstores.head())
print(dfstores.shape[0])
print(dffeatures.head())
print(dffeatures.shape[0])

   Store Type    Size
0      1    A  151315
1      2    A  202307
2      3    B   37392
3      4    A  205863
4      5    B   34875
45
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211.289143         8.106      False  
3        NaN        NaN        NaN  211.319643         8.106      False  
4        NaN        NaN        NaN  211.350143         8.106      Fa

In [9]:
#merge the features in the dataframes
dftrain = pd.merge(dftrain, dfstores, on='Store', how='left')
dftrain = pd.merge(dftrain, dffeatures, on=['Store', 'Date'], how='left')
dftest = pd.merge(dftest, dfstores, on='Store', how='left')
dftest = pd.merge(dftest, dffeatures, on=['Store', 'Date'], how='left')

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday_x  Seasonality  Year Type  \
0      1     1  2010-02-05      24924.50        False            5  2010    A   
1      1     1  2010-02-12      46039.49         True            6  2010    A   
2      1     1  2010-02-19      41595.55        False            7  2010    A   
3      1     1  2010-02-26      19403.54        False            8  2010    A   
4      1     1  2010-03-05      21827.90        False            9  2010    A   

     Size  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
0  151315        42.31       2.572        NaN        NaN        NaN   
1  151315        38.51       2.548        NaN        NaN        NaN   
2  151315        39.93       2.514        NaN        NaN        NaN   
3  151315        46.63       2.561        NaN        NaN        NaN   
4  151315        46.50       2.625        NaN        NaN        NaN   

   MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday_y  
0        NaN        N

In [0]:
#validate that IsHoliday is consistent in the dataframes
for index, row in dftrain.iterrows():
  if row['IsHoliday_x'] != row['IsHoliday_y']:
    print(row)
for index, row in dftest.iterrows():
  if row['IsHoliday_x'] != row['IsHoliday_y']:
    print(row)


In [11]:
#IsHoliday is duplicated, remove one
dftrain.drop("IsHoliday_y", axis=1, inplace=True) 
dftest.drop("IsHoliday_y", axis=1, inplace=True) 

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday_x  Seasonality  Year Type  \
0      1     1  2010-02-05      24924.50        False            5  2010    A   
1      1     1  2010-02-12      46039.49         True            6  2010    A   
2      1     1  2010-02-19      41595.55        False            7  2010    A   
3      1     1  2010-02-26      19403.54        False            8  2010    A   
4      1     1  2010-03-05      21827.90        False            9  2010    A   

     Size  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
0  151315        42.31       2.572        NaN        NaN        NaN   
1  151315        38.51       2.548        NaN        NaN        NaN   
2  151315        39.93       2.514        NaN        NaN        NaN   
3  151315        46.63       2.561        NaN        NaN        NaN   
4  151315        46.50       2.625        NaN        NaN        NaN   

   MarkDown4  MarkDown5         CPI  Unemployment  
0        NaN        NaN  211.09635

In [15]:
#Save the dataframes in csv files in Google Drive
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/drive/My Drive/"
dftrain.to_csv('/content/drive/My Drive/train.csv', na_rep='NA', index=False)
dftest.to_csv('/content/drive/My Drive/test.csv', na_rep='NA',index=False)
!ls "/content/drive/My Drive/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
'Colab Notebooks'	       test.csv   'Training Evaluation Form.gform'
'it works pretty well!.gdoc'   train.csv
'Colab Notebooks'	       test.csv   'Training Evaluation Form.gform'
'it works pretty well!.gdoc'   train.csv


In [0]:
#Save the dataframes in csv files (locally in the VM)
dftrain.to_csv('train.csv', na_rep='NA', index=False)
dftest.to_csv('test.csv', na_rep='NA', index=False)

In [0]:
#Download the csv files
from google.colab import files
files.download('train.csv')
files.download('test.csv')

#added to github to the data folder

#that's it for now!